In [2]:
# import all libraries you need here
import pandas as pd
import numpy as np
import pathlib as pl

import scanpy as sc

# Step 0: Download the data

In [3]:
path_data = pl.Path("/Users/leonfroehlich/Developer/Machine Learning for Genomics/Project 2/data")

In [4]:
all_bulkified = pd.read_csv(path_data / "test_data/test_bulk.csv",index_col=0)
train_adata = sc.read_h5ad(path_data / "train_data/train_adata.h5ad")
test_adata = sc.read_h5ad(path_data / "test_data/test_adata.h5ad")

print(train_adata.var)

            n_cells     mt  n_cells_by_counts  mean_counts  \
Unnamed: 0                                                   
NOC2L          6735  False               6735     0.143681   
HES4           8287  False               8287     0.330635   
ISG15         19640  False              19640     0.871934   
TNFRSF18      12021  False              12021     0.682345   
TNFRSF4        7880  False               7880     0.484510   
...             ...    ...                ...          ...   
MT-ND6        15837   True              15837     0.453242   
MT-CYB        52022   True              52022    22.234161   
SELE           1192  False               1192     0.330941   
SOX17          1125  False               1125     0.051330   
CCL14          1423  False               1423     0.057266   

            pct_dropout_by_counts  total_counts  
Unnamed: 0                                       
NOC2L                   87.143757        7527.0  
HES4                    84.181190       173

In [5]:
print(f"Number of patients to deconvolve: {all_bulkified.shape[1]}")
print(f"Number of genes in dataset: {all_bulkified.shape[0]}")

Number of patients to deconvolve: 20
Number of genes in dataset: 7725


In [6]:
print(f"Number of cells in the train set {train_adata.n_obs}")
for spl in train_adata.obs.Sample.unique():
    print(f"Number of cells for {spl} is {train_adata[train_adata.obs.Sample==spl].n_obs}")

Number of cells in the train set 32374
Number of cells for s1 is 6821
Number of cells for s2 is 4555
Number of cells for s3 is 5166
Number of cells for s4 is 5607
Number of cells for s7 is 4471
Number of cells for s8 is 5754


In [7]:
print(f"There are {train_adata.obs.highLevelType.nunique()} different cell types in the dataset")
print(f"The different cells types are {train_adata.obs.highLevelType.unique().astype(str)}")

There are 9 different cell types in the dataset
The different cells types are ['T' 'B' 'Plasmablast' 'Fibroblast' 'Mast' 'Myeloid' 'NK' 'Myofibroblast'
 'Endothelial']


In [8]:
print(f"Number of cells in the test set {test_adata.n_obs}")
for spl in test_adata.obs.Sample.unique():
    print(f"Number of cells for {spl} is {test_adata[test_adata.obs.Sample==spl].n_obs}")

Number of cells in the test set 18616
Number of cells for s5 is 6020
Number of cells for s6 is 5530
Number of cells for s9 is 3336
Number of cells for s10 is 3730


# Step 1: Deconvolve the data

In [9]:
# Write code here to deconvolve the data of all bulkified to obtain estimates of cell type proportions
from scipy.optimize import nnls 

bulk_data = pd.read_csv(path_data / "train_data/train_bulk.csv",index_col=0)
bulk_data_true = pd.read_csv(path_data / "train_data/train_bulk_trueprops.csv",index_col=0)

print(bulk_data.shape)
print(bulk_data_true.shape)

B = bulk_data.values
P = bulk_data_true.values

P_pinv = np.linalg.pinv(P)

E = B @ P_pinv

T = all_bulkified.values
num_samples = T.shape[1]
num_ct = E.shape[1]

P_estimated = np.zeros((num_ct, num_samples))

for i in range(num_samples):
    b = T[:, i]
    p, _ = nnls(E, b)
    s = p.sum()
    if s > 0:
        p = p / s
    P_estimated[:, i] = p


print(P_estimated.shape)


(7725, 12)
(9, 12)
(9, 20)


# Step 2: Perform clustering 

In [10]:
adata = train_adata.copy()

# preprocessing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata.raw = adata
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)

# dimensionality reduction
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

sc.tl.leiden(adata, resolution=0.5)

# map clusters to cell types
cluster_to_celltype = {}
for cluster in adata.obs['leiden'].unique():
    cells_in_cluster = adata[adata.obs['leiden'] == cluster]
    most_common_celltype = cells_in_cluster.obs['highLevelType'].value_counts().idxmax()
    cluster_to_celltype[cluster] = most_common_celltype

adata.obs['predicted'] = adata.obs['leiden'].map(cluster_to_celltype)

true_labels = adata.obs['highLevelType']
predicted_labels = adata.obs['predicted']

n_correct = (true_labels == predicted_labels).sum()
n_total = len(true_labels)
print(f"Clustering accuracy: {n_correct}/{n_total} = {n_correct/n_total:.4f}")

/Users/leonfroehlich/miniconda3/envs/ml4gProj2/lib/python3.11/site-packages/scanpy/preprocessing/_scale.py:309: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/leonfroehlich/miniconda3/envs/ml4gProj2/lib/python3.11/functools.py:909: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)
/Users/leonfroehlich/miniconda3/envs/ml4gProj2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/2h/ypppzt45267dz6xnbvl4sp280000gn/T/ipykernel_54274/1660406450.py:17: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.


Clustering accuracy: 29925/32374 = 0.9244


# Step 2bis: Predict on the test data

In [11]:
# Use the same clustering technique to cluster the test data
from sklearn.ensemble import RandomForestClassifier

train = train_adata.copy()
sc.pp.normalize_total(train, target_sum=1e4)
sc.pp.log1p(train)
sc.pp.highly_variable_genes(train, n_top_genes=3000)

X_train = train[:, train.var.highly_variable]
Y_train = train.obs['highLevelType']

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train.X, Y_train)

test = test_adata.copy()
sc.pp.normalize_total(test, target_sum=1e4)
sc.pp.log1p(test)
X_test = test[:, train.var.highly_variable]

X_test.obs['predicted'] = clf.predict(X_test.X)


/var/folders/2h/ypppzt45267dz6xnbvl4sp280000gn/T/ipykernel_54274/3652520257.py:20: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  X_test.obs['predicted'] = clf.predict(X_test.X)


# Step 3: Save the required files

In [12]:
# pred_props should be a DataFrame containing the estimated cell type proportions for the patients in all_bulkified
# pred_props.columns = ['index','s5_0','s5_1',...,'s10_3','s10_4'] = np.append(["index"],all_bulkified.columns)
# pred_props['index'] =  

cell_types = ['T', 'Endothelial', 'Fibroblast', 'Plasmablast', 'B', 'Myofibroblast', 'NK', 'Myeloid', 'Mast']

pred_props = pd.DataFrame(P_estimated, index=cell_types, columns=all_bulkified.columns)

pred_props.insert(0, 'index', cell_types)

print(pred_props.shape)
print(pred_props)
#save pred_props to csv
pred_props.to_csv("pred_props.csv", index=False)

(9, 21)
                       index      s5_0      s5_1      s5_2      s5_3  \
T                          T  0.105103  0.498989  0.438723  0.406354   
Endothelial      Endothelial  0.000000  0.072899  0.059647  0.055309   
Fibroblast        Fibroblast  0.000000  0.000000  0.000000  0.000000   
Plasmablast      Plasmablast  0.142559  0.047065  0.063201  0.079515   
B                          B  0.000000  0.012215  0.011854  0.004380   
Myofibroblast  Myofibroblast  0.537366  0.247422  0.287958  0.306430   
NK                        NK  0.089495  0.069981  0.063107  0.074142   
Myeloid              Myeloid  0.065935  0.044114  0.046097  0.051268   
Mast                    Mast  0.059543  0.007315  0.029413  0.022601   

                   s5_4      s6_0      s6_1      s6_2      s6_3  ...  \
T              0.487963  0.533413  0.560869  0.546053  0.537088  ...   
Endothelial    0.068844  0.074517  0.077954  0.077386  0.076729  ...   
Fibroblast     0.000000  0.000000  0.000000  0.000000  

In [13]:
results_path = pl.Path("results")

In [14]:
assert all(pred_props.columns == np.append(["index"],all_bulkified.columns)), "Wrong columns"

In [15]:
assert all(pred_props['index']== ['T', 'Endothelial', 'Fibroblast', 'Plasmablast', 'B', 'Myofibroblast',
       'NK', 'Myeloid', 'Mast']), "Wrong order for cell types"

In [16]:
assert all(pred_props.drop("index",axis=1).sum().round()==1), "The proportions for a single patient must sum to 1"

In [17]:
# cluster_labels should be a DataFrame containing the cluster labels for each cell
# cluster_labels.columns = ["index", "cluster"]
# cluster_labels["index"] = test_adata.columns

cluster_labels = pd.DataFrame({
    "index": X_test.obs_names,
    "cluster": X_test.obs['predicted'].astype(str)
})

print(cluster_labels)
print(cluster_labels.shape)
#save cluster_labels to csv
cluster_labels.to_csv("cluster_membership.csv", index=False)

                                       index     cluster
AAACCCAAGGAGGCAG-1_5    AAACCCAAGGAGGCAG-1_5           T
AAACCCAAGTTGCGCC-1_5    AAACCCAAGTTGCGCC-1_5           T
AAACCCACACGGATCC-1_5    AAACCCACACGGATCC-1_5           T
AAACCCACATCGGAAG-1_5    AAACCCACATCGGAAG-1_5           T
AAACCCAGTGCGAGTA-1_5    AAACCCAGTGCGAGTA-1_5           T
...                                      ...         ...
TTTGGTTCATTGAAGA-1_10  TTTGGTTCATTGAAGA-1_10           T
TTTGGTTGTTGTCCCT-1_10  TTTGGTTGTTGTCCCT-1_10  Fibroblast
TTTGGTTGTTTGACAC-1_10  TTTGGTTGTTTGACAC-1_10          NK
TTTGTTGAGGGTCAAC-1_10  TTTGTTGAGGGTCAAC-1_10           T
TTTGTTGCATGGAGAC-1_10  TTTGTTGCATGGAGAC-1_10  Fibroblast

[18616 rows x 2 columns]
(18616, 2)


In [18]:
assert all(cluster_labels.columns == ["index", "cluster"]), "Wrong columns"

In [19]:
assert all(cluster_labels["index"] == test_adata.obs_names), "The cell ids are either not all present or not in the right order"

In [20]:
import zipfile
from pathlib import Path

archive_name = "Froehlich_Leon_Project2.zip" # TODO
env_file = Path("environment.yml")

with zipfile.ZipFile(results_path / archive_name, "x") as zf:
    with zf.open(f"pred_props.csv", "w") as buffer:
        pred_props.to_csv(buffer)
    with zf.open(f"cluster_membership.csv", "w") as buffer:
        cluster_labels.to_csv(buffer)
    zf.close()

with zipfile.ZipFile(results_path / "SourceCode.zip", "x") as zf:
    zf.write("Project2_starter_code.ipynb", arcname="Project2_starter_code.ipynb")
    zf.write(env_file, arcname="environment.yml")
    zf.close()